### A full training loop

We will now look at a 2-class classification problem with a generated toy-dataset.
The data will be stored inside a PyTorch `Dataset` subclass, which will provide you with some utilities like automatic shuffling and batching for the training loop, if paired with `DataLoader`.

The 2 classes are two normal distributions with different means.

In [ ]:
%pip install matplotlib # install using pip
#%conda install -y matplotlib # or install using conda

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt

class TwoClassDataset(Dataset):
    def __init__(self):
        # create synthetic dataset
        self.features_a = torch.normal(mean=1., std=1.0, size=(100,2))
        self.labels_a = torch.zeros(100, dtype=torch.long)
        self.features_b = torch.normal(mean=-1.0, std=1.0, size=(100,2))
        self.labels_b = torch.ones(100, dtype=torch.long)

        self.data = torch.cat((self.features_a, self.features_b), dim=0) # X: 200 x 2 matrix : [x1,x2]
        self.labels = torch.cat((self.labels_a, self.labels_b), dim=0) # Y: binary vector of length 200: 0: class A, 1: class B
    
    def __len__(self):
        return self.data.size(dim=0)
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
    
train_data = TwoClassDataset()

# plot both classes
fig, ax = plt.subplots()
ax.scatter(train_data.features_a[:,0], train_data.features_a[:,1], label="class 1", c="darkorange")
ax.scatter(train_data.features_b[:,0], train_data.features_b[:,1], label="class 2", c="dodgerblue")
ax.legend()
plt.show()

In [ ]:
print("first sample input, label:", train_data[0])

The data loader can shuffle your dataset for each epoch (which you should definitely do to keep mini-batches representative) and also automatically groups your single samples together into matrices for a mini-batch (the first dimension will always be the batch dimension, e.g. if your data point has size $3$, then a batch of size 25 would have size $25 \times 3$.

Take a look at the basics about dataloaders and datasets: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

How can we create a DataLoader for our `TwoClassDataset` with a batch size of 5 and shuffling enabled?
- Print a sample from the dataset
- Print the training data samples and labels separately

In [ ]:
# create a DataLoader

# print data and labels 

We're creating a new clasifier for our problem, the input size is 2 and the output size is 2 because we want to do multi-class classification (for 2 classes, a sigmoid and a single output would also work).
The activation function is a ReLU (rectified linear unit). It is piecewise linear: $relu(x) = max(0, x)$

In [ ]:
class Classifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # define the layers
        
    def forward(self, x):
        # define the forward pass

In [ ]:
%pip install tqdm
#%conda install -y tqdm # or install using conda

For the training loop, we need instances of
* the training data (in real applications, also development / test data)
* a loss criterion (e.g. MSE for regression tasks, Cross Entropy for classification)
* an optimizer

We will iterate many times over the full dataset (each iteration is called an *epoch*).
Each epoch is broken up further into mini-batches that can be adapted to fit e.g. your computer's memory constraints.

For each mini-batch, we need to:
* reset the gradients (otherwise, they will accumulate)
* forward the input data to create predictions
* calculate the loss between the predictions and the gold labels
* calculate the gradient of the loss w.r.t. the network weights (backward pass)
* update the network weights using the gradients and the optimizer

Please implement the following steps to complete the training loop:

1. **Reset gradients to 0**
   - Task: Reset the gradients to zero using the optimizer.
   - Variable: `optim`

2. **Access data and labels from batch**
   - Task: Access the input data and labels from the current mini-batch.
   - Variables: `inputs`, `gold_labels`

3. **Perform the forward pass**
   - Task: Perform the forward pass to get the predictions from the model.
   - Variables: `predictions`, `model`

4. **Calculate the loss and perform the backward pass**
   - Task: Calculate the loss and perform the backward pass to compute the gradients.
   - Variables: `loss`, `loss_func`

5. **Update the network weights**
   - Task: Update the network weights using the optimizer.
   - Variable: `optim`

In [ ]:
from tqdm import tqdm

# instantiate model, loss criterion and optimizer
classy = Classifier()
loss_func = torch.nn.CrossEntropyLoss()
optim = torch.optim.SGD(params=classy.parameters(), lr=0.01)

# store epoch metrics
epoch_accs = []
epoch_losses = []
dataloader = DataLoader(dataset=train_data, batch_size=25, shuffle=True)

# epoch loop
for epoch in tqdm(range(50)):
    epoch_acc = []
    epoch_loss = []
    
    # mini-batch loop for one epoch
    for batch in dataloader:
        # reset gradients to 0
        
        # access data and labels from batch

        # forward pass
       
        # loss and backward pass
        
        # update network weights
        
        # check accuracy (get predicted class for each sample, compare to gold label)
        category_probs = torch.softmax(predictions, dim=1) # sums up to 1 for each sample
        category_labels = torch.argmax(category_probs, dim=1) # extract most likely label
        batch_acc = (category_labels == gold_labels).float().sum(dim=0)/25.0 # avg accuracy for batch
        epoch_acc.append(batch_acc.item())
        epoch_loss.append(loss.item())

    # average all metrics across one epoch
    epoch_losses.append(sum(epoch_loss)/len(epoch_loss))
    epoch_accs.append(sum(epoch_acc)/len(epoch_acc))
  


In [ ]:
plt.plot(epoch_accs)

In [ ]:
plt.plot(epoch_losses)

We'll visualize the predictions for an unseen test dataset drawn from the same distribution as the training data. Red points mark misclassified samples.

In [ ]:
from matplotlib import pyplot as plt

test_data = TwoClassDataset()

predicted_class_a = []
predicted_class_b = []
missclassified_class_a = []
missclassified_class_b = []
for i in range(len(test_data)):
    x, label = test_data[i]
    pred = classy(x)
    probs = torch.softmax(pred, dim=0) # note that the softmax is mathematically not needed since it is a monotonic function
    class_pred = torch.argmax(probs, dim=0)
  
    if label != class_pred:
        if label == 0:
            missclassified_class_a.append(x.unsqueeze(dim=0))
        else:
            missclassified_class_b.append(x.unsqueeze(dim=0))
    else:
        if class_pred == 0:
            predicted_class_a.append(x.unsqueeze(dim=0))
        else:
            predicted_class_b.append(x.unsqueeze(dim=0))
predicted_class_a = torch.cat(predicted_class_a, dim=0) # concatenates tensor along specified, existing dimension, does not create new dimension
predicted_class_b = torch.cat(predicted_class_b, dim=0) # `stack` is an alternative which creates a new dimension (i.e. we wouldn't have `unsqueeze`d the vectors)
missclassified_class_a = torch.cat(missclassified_class_a, dim=0)
missclassified_class_b = torch.cat(missclassified_class_b, dim=0)

fig, ax = plt.subplots()
ax.scatter(predicted_class_a[:,0], predicted_class_a[:,1], label="class 1", c="darkorange")
ax.scatter(predicted_class_b[:,0], predicted_class_b[:,1], label="class 2", c="dodgerblue")
ax.scatter(missclassified_class_a[:,0], missclassified_class_a[:,1], c="darkorange", edgecolors="r")
ax.scatter(missclassified_class_b[:,0], missclassified_class_b[:,1], c="dodgerblue", edgecolors="r")
ax.legend()
plt.show()